In [60]:
# Referencias Web Scrapping

#JavaScript

#https://www.youtube.com/watch?v=vcnomT0CP0Y 
#https://medium.com/@henriquecoura_87435/web-scraping-com-python-selenium-e-javascript-faa108f95bbe
#https://kiwidamien.github.io/webscraping-beyond-beautifulsoup-and-selenium.html
#https://www.dataquest.io/blog/web-scraping-beautifulsoup/
#https://stackoverflow.com/questions/45448994/wait-page-to-load-before-getting-data-with-requests-get-in-python-3
#https://www.codementor.io/blog/python-web-scraping-63l2v9sf2q


In [61]:
#!pip install html5lib
#!pip install selenium

In [62]:
import requests
import os
import numpy as np
import pandas as pd
import html5lib
from bs4 import BeautifulSoup
from selenium import webdriver

import time
from random import randint
import warnings

import unicodedata
import re
from selenium.webdriver.support.ui import WebDriverWait

In [63]:
def get_total_houses(driver, total_houses_initial):
    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(3)
    search_h1 = soup.find('h1') 
    total_houses_real = search_h1.strong.text
    total_houses_real = pd.to_numeric(total_houses_real)
    print(f"#1 -> # of houses: initial = {total_houses_initial}  real = {total_houses_real}")
    return total_houses_real != total_houses_initial

"""
A remoção de acentos foi baseada em uma resposta no Stack Overflow.
http://stackoverflow.com/a/517974/3464573
"""

def removerAcentosECaracteresEspeciais(palavra):

    # Unicode normalize transforma um caracter em seu equivalente em latin.
    nfkd = unicodedata.normalize('NFKD', palavra)
    palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])

    # Usa expressão regular para retornar a palavra apenas com números, letras e espaço
    return re.sub('[^a-zA-Z0-9 \\\]', '', palavraSemAcento)

In [64]:
## corrigir captura de valor do condomínio
## pegar os atributos 

houses_list= []
source_urls= []
titles = []
urls= []
addresses = []
areas = []
n_bedrooms = []
n_en_suites = []
prices = []
n_bathrooms = []
n_parking_spaces = []
condo_charges = []
page_n=[]
house_n=[]

fireFoxOptions = webdriver.FirefoxOptions()
fireFoxOptions.add_argument('--ignore-certificate-errors')
fireFoxOptions.add_argument('--incognito')
#fireFoxOptions.add_argument('--headless')
fireFoxOptions.add_argument('--no-sandbox')
fireFoxOptions.add_argument('--disable-dev-shm-usage')

driver = webdriver.Firefox(firefox_options=fireFoxOptions)


# Preparing the monitoring of the loop
start_time = time.time()
requests_ = 0

#Looping for the searches
for search in range(5): 

        
    # Aldeia da serra: Santana de Parnaiba e Barueri
    if search == 0:
        source_url = "https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/"\
       "?pagina=2#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Aldeia_da_Serra,BR-Sao_Paulo-NULL-"\
       "Santana_de_Parnaiba-Barrios-Aldeia_da_Serra"

    #Pinheiros e Flores    
    if search == 1:
        source_url = "https://www.vivareal.com.br/venda/sp/santana-de-parnaiba/bairros/"\
        "morada-dos-pinheiros-aldeia-da-serra/?__vt=plp:b&pagina=2#onde=BR-Sao_Paulo-NULL"\
        "-Santana_de_Parnaiba-Barrios-Morada_dos_Pinheiros_Aldeia_da_Serra,BR-Sao_Paulo-NULL"\
        "-Santana_de_Parnaiba-Barrios-Morada_das_Flores_Aldeia_da_Serra"

    #Estrelas e Lagos
    if search == 2:
        source_url = "https://www.vivareal.com.br/venda/sp/barueri/bairros/residencial-morada-dos-lagos/"\
        "?__vt=plp:b&pagina=2#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Residencial_Morada_dos_Lagos"\
        ",BR-Sao_Paulo-NULL-Barueri-Barrios-Praca_da_Aldeia_da_Serra_I"

    #Pássaros, Praça da Aldeia e Quintas do Ingaí
    if search == 3:
        source_url = "https://www.vivareal.com.br/venda/sp/barueri/bairros/morada-dos-passaros/"\
        "?__vt=plp:b&pagina=2#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Morada_dos_Passaros"\
        ",BR-Sao_Paulo-NULL-Barueri-Barrios-Praca_da_Aldeia_da_Serra_I,BR-Sao_Paulo-NULL-Santana_de_"\
        "Parnaiba-Barrios-Quintas_do_Ingai"

    # Estrada Dr Yojiro Takaoka - Morada da Serra
    if search == 4:
        source_url = "https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/"\
        "estrada-doutor-yojiro-takaoka/condominio_residencial/?__vt=plp:b#onde=BR-Sao_Paulo"\
        "-NULL-Barueri-Barrios-Aldeia_da_Serra|Estrada_Doutor_Yojiro_Takaoka"
        
    print(source_url) 
    
    #Access webpage with search url
    driver.get(source_url)
    time.sleep(2.5)
    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(2.5)
    #Get initial number of total houses returned from the search 
    search_h1 = soup.find('h1') 
    total_houses_initial = search_h1.strong.text
    total_houses_initial = pd.to_numeric(total_houses_initial)
    print("#0") 
    
    # Loop until get the real total number of houses: Java Script load total pages after few seconds    
    WebDriverWait(driver, 25).until(lambda driver: get_total_houses(driver, total_houses_initial))    
    print("#2")
      
    #Get the real total number of houses
    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(2.5) 
    search_h1 = soup.find('h1') 
    total_houses_real = search_h1.strong.text
    total_houses_real = pd.to_numeric(total_houses_real)
    number_of_pages = np.ceil((total_houses_real/36)).astype(int)
   
       
    # For every page in the search
#    for page in range(3):
    for page in range(number_of_pages):
        
        pagina = str(page+1)
        
        
         # Choose search 1 first and search 2 on the 2nd iteration

        # Aldeia da serra: Santana de Parnaiba e Barueri           
        if search == 0:
            source_url_b = "https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/"\
            "?pagina="+pagina+"#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Aldeia_da_Serra,BR-Sao_Paulo-NULL-"\
            "Santana_de_Parnaiba-Barrios-Aldeia_da_Serra"
            
        #Pinheiros e Flores    
        if search == 1:
            source_url_b = "https://www.vivareal.com.br/venda/sp/santana-de-parnaiba/bairros/"\
            "morada-dos-pinheiros-aldeia-da-serra/?__vt=plp:b&pagina="+pagina+"#onde=BR-Sao_Paulo-NULL"\
            "-Santana_de_Parnaiba-Barrios-Morada_dos_Pinheiros_Aldeia_da_Serra,BR-Sao_Paulo-NULL"\
            "-Santana_de_Parnaiba-Barrios-Morada_das_Flores_Aldeia_da_Serra"

        #Estrelas e Lagos
        if search == 2:
            source_url_b = "https://www.vivareal.com.br/venda/sp/barueri/bairros/residencial-morada-dos-lagos/"\
            "?__vt=plp:b&pagina="+pagina+"#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Residencial_Morada_dos_Lagos"\
            ",BR-Sao_Paulo-NULL-Barueri-Barrios-Praca_da_Aldeia_da_Serra_I"

        #Pássaros, Praça da Aldeia e Quintas do Ingaí
        if search == 3:
            source_url_b = "https://www.vivareal.com.br/venda/sp/barueri/bairros/morada-dos-passaros/"\
            "?__vt=plp:b&pagina="+pagina+"#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Morada_dos_Passaros"\
            ",BR-Sao_Paulo-NULL-Barueri-Barrios-Praca_da_Aldeia_da_Serra_I,BR-Sao_Paulo-NULL-Santana_de_"\
            "Parnaiba-Barrios-Quintas_do_Ingai"

        # Estrada Dr Yojiro Takaoka - Morada da Serra
        if search == 4:
            source_url_b = "https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/"\
            "estrada-doutor-yojiro-takaoka/condominio_residencial/?__vt=plp:b#onde=BR-Sao_Paulo"\
            "-NULL-Barueri-Barrios-Aldeia_da_Serra|Estrada_Doutor_Yojiro_Takaoka"
            
            
        print(f"pagina: {str(page+1)}")
        #print(source_url_b)
              
        #Get the search html for one page
        driver.get(source_url_b)
        time.sleep(2.5)
        html = driver.execute_script("return document.documentElement.outerHTML")
        soup = BeautifulSoup(html, 'html.parser') 
        print("#3")
        
        # Get total number of pages: Java Script load total pages after few seconds    
        WebDriverWait(driver, 25).until(lambda driver: get_total_houses(driver, total_houses_initial))    
        print("#4")
        time.sleep(2.5)
        html = driver.execute_script("return document.documentElement.outerHTML")
        soup = BeautifulSoup(html, 'html.parser')
        time.sleep(2.5)
        search_h1 = soup.find('h1') 
        total_houses_real = search_h1.strong.text
        total_houses_real = pd.to_numeric(total_houses_real)

#         print(f"{total_houses_initial}: initial houses and {total_houses_real} total houses")

        print("#5")

        # Parse the content of the request with BeautifulSoup
        search_h1 = soup.find('h1') 
        total_houses = search_h1.strong.text

        # Select all the 36 house containers from a single page
        houses_list = soup.find_all('div', attrs = {'class':'property-card__main-content'})

        #print(type(houses_list))
        houses_per_page = len(houses_list)     
        house = 1    

#         print("#6")            
        for house in range(houses_per_page):
            #time.sleep(0.05)
            #print(f"house {house}, de houses{houses_per_page}")
            page_n.append(page)
            house_n.append(house)
            
            title = houses_list[house].h2.a.text
            titles.append(title)

    #         if houses_list[house].h2.a:           
    #             title = houses_list[house].h2.a.text
    #         else: title=''
    #         titles.append(title)        

            if houses_list[house].h2.a['href']:           
                url = 'www.vivareal.com.br'+houses_list[house].h2.a['href']
            else: url=''
            urls.append(url)

            if houses_list[house].h2.span:
                address = houses_list[house].h2.span.text
            else: address = ''
            addresses.append(address)         

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-area'):
                area_m2 = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-area').text
            else: 
                area_m2 = '0'
            areas.append(area_m2)

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-room js-property-detail-rooms'):
                bedrooms = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-room js-property-detail-rooms').text
            else: 
                bedrooms = '0'
            n_bedrooms.append(bedrooms)        

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-item-extra js-property-detail-suites'):
                en_suites = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-item-extra js-property-detail-suites').text
            else: en_suites = '0'
            n_en_suites.append(en_suites)

            if houses_list[house].section.div:
                price = houses_list[house].section.div.text
            else: price = '0'
            prices.append(price)

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'):
                bathrooms = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom').text
            else: bathrooms='0'
            n_bathrooms.append(bathrooms)

            if houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-garage js-property-detail-garages'):
                parking_spaces = houses_list[house].find('li', class_ = 'property-card__detail-item property-card__detail-garage js-property-detail-garages').text
            else: parking_spaces='0'
            n_parking_spaces.append(parking_spaces)        

            house += 1

            #x = houses_list[h].section.footer.strong.text
            #x = houses_list[h].find('div', class_ = "property-card__price-details--condo").text
            #houses_list[h].section.footer.strong.text
            #print(x)

    page += 1


<ipython-input-64-3a86b81ae40e>:26: DeprecationWarning: use options instead of firefox_options
  driver = webdriver.Firefox(firefox_options=fireFoxOptions)


https://www.vivareal.com.br/venda/sp/barueri/bairros/aldeia-da-serra/?pagina=2#onde=BR-Sao_Paulo-NULL-Barueri-Barrios-Aldeia_da_Serra,BR-Sao_Paulo-NULL-Santana_de_Parnaiba-Barrios-Aldeia_da_Serra
#0
#1 -> # of houses: initial = 517  real = 815
#2
pagina: 1
#3
#1 -> # of houses: initial = 517  real = 815
#4
#5
pagina: 2
#3
#1 -> # of houses: initial = 517  real = 815
#4
#5
pagina: 3
#3
#1 -> # of houses: initial = 517  real = 517
#1 -> # of houses: initial = 517  real = 815
#4
#5
pagina: 4
#3
#1 -> # of houses: initial = 517  real = 517
#1 -> # of houses: initial = 517  real = 815
#4
#5
pagina: 5
#3
#1 -> # of houses: initial = 517  real = 517
#1 -> # of houses: initial = 517  real = 815
#4
#5
pagina: 6
#3
#1 -> # of houses: initial = 517  real = 517
#1 -> # of houses: initial = 517  real = 815
#4
#5
pagina: 7
#3
#1 -> # of houses: initial = 517  real = 517
#1 -> # of houses: initial = 517  real = 815
#4
#5
pagina: 8
#3
#1 -> # of houses: initial = 517  real = 517
#1 -> # of houses: ini

In [65]:
driver.close()

In [83]:
aldeia_imoveis = pd.DataFrame({ 'page_seq': page_n,
                                'house_seq': house_n,
                                'title': titles,
                                'url': urls,
                                'address': addresses,
                                'area_m2': areas,
                                'bedrooms': n_bedrooms,
                                'en_suites': n_en_suites,
                                'price': prices,
                                'bathrooms': n_bathrooms,
                                'parking_spaces': n_parking_spaces
                               
                            })
print(aldeia_imoveis.info())
#aldeia_imoveis.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2124 entries, 0 to 2123
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   page_seq        2124 non-null   int64 
 1   house_seq       2124 non-null   int64 
 2   title           2124 non-null   object
 3   url             2124 non-null   object
 4   address         2124 non-null   object
 5   area_m2         2124 non-null   object
 6   bedrooms        2124 non-null   object
 7   en_suites       2124 non-null   object
 8   price           2124 non-null   object
 9   bathrooms       2124 non-null   object
 10  parking_spaces  2124 non-null   object
dtypes: int64(2), object(9)
memory usage: 182.7+ KB
None


In [84]:
aux = aldeia_imoveis.copy()
#aux =  aux[ aux['title'].str.contains('Casa') | aux['title'].str.contains('Sobrado')]

In [85]:
aux.to_csv('../output/imoveis_aldeia.csv', index=None)